In [2]:
#for parsing Genbank files.  Does these things:
#finds and reads a gb file stored on local drive
#makes a list of gene orientations, names, start, end, size and a dummy variable
#makes a codon usage table, but it ignores start and stop codons
#plots codon numbers vs. codon frequencies -- should give a straight line
#allows searches for genes

genomeFileName = '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/P.fluorescens.sequence.gb'
genesname =      '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/P.fluorescens.sequence.Gene.Database.txt'

import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive

drive.mount('/content/gdrive')
with open(genomeFileName, 'r') as gbfile:
  data = gbfile.read()
gbfile.close()
txtfile = str(data)
start = 3
stop = 3
frame = 0

# COLLECTS ALL THE GENES IN A LIST CALLED "features" AND THE SEQUENCE WILL BE IN split2[1]

split1 = txtfile.split('FEATURES') # split1[0] is annotations, split1[1] is the good stuff
split2 = split1[1].split('ORIGIN')  # gene locations are in split2[0] and genome sequence is in split2[1]
features = split2[0].split('  gene  ')  # separates each gene location and puts them in a list

seq = ''
for char in split2[1]:                  #puts sequence in a single, long string
    if char in ('t', 'g', 'c', 'a'):
        seq += char
        
print('The number of genes in "Features" =', len(features) -1)
print('The "sub-1" Features line starts with:', features[1][:100])
print('The first 100 nucleotides =', seq[:100])        

#COLLECT ALL THE FORWARD AND REVERSE GENES and labels them as 'FOR' or 'REV'
names =[]
forward =0
reverse =0

for x in range(1, len(features)):
  if 'join' not in features[x]:
    if 'complement' not in features[x]: genedirection = 'FOR'
    elif 'complement' in features[x]:   genedirection = 'REV'
    cleanfeatures = features[x].replace('complement(','').replace(')', '')\
    .replace('/', '').replace(' ', '').replace('gene="', '').replace('"', '')
    workgene = cleanfeatures.split('\n')
    genename = workgene[1]
    genelocation = workgene[0].split('.')
    genestart = genelocation[0] 
    geneend = genelocation[2] 
    genedata = [genedirection, genename, genestart.replace('>', '').replace('<', ''), geneend.replace('<', '').replace('>',''), 0, 0.0]
    names.append(genedata)
              
#converts gene starts and ends to integers.  Calulates gene length
print('The number of annotated genes =', len(names))
for y in range(len(names)):
  try:
    names[y][2] = int(names[y][2])
    names[y][3] = int(names[y][3])
    names[y][4] = int(names[y][3]) - int(names[y][2])
  except: print('Bad gene:', names[y])
                  
genes =[]
for x in range(len(names)):
  if names[x][0] == 'FOR':
    cds = seq[names[x][2] -1 : names[x][3]]
  elif names[x][0] == 'REV':
    transposed = ''
    flipped = seq[names[x][2] -1 : names[x][3]]
    for char in flipped:
      if char == 'a': newchar = 't'
      elif char == 't': newchar = 'a'
      elif char == 'c': newchar = 'g'
      elif char == 'g': newchar = 'c'
      transposed += newchar
    cds = transposed[::-1]
  genes.append(cds)

#collects only those sequences that have proper start codons
startgenes =[]
startnames =[]
for x in range(len(genes)):
  firstcodon = genes[x][0:3]
  if firstcodon in ('atg', 'gtg', 'ctg', 'ttg', 'att', 'ata', 'aaa'):
    startgenes.append(genes[x])
    startnames.append(names[x])

#Collects only those sequences that do not contain internal stop codons
goodgenes =[]
goodnames =[]
for x in range(len(startgenes)):
  delete = False
  for a in range(start, len(startgenes[x]) - stop):
    if a % 3 == frame and delete == False:
      codon = startgenes[x][a:a+2] 
      if codon in ('tag', 'taa', 'tga'):
        delete = True
  if delete == False : 
    goodgenes.append(startgenes[x])
    goodnames.append(startnames[x])    

print(goodnames[0])
print(goodgenes[0])

outgenes =[]
for x in range(len(goodgenes)):
  gene = goodnames[x][1] +','+ str(goodnames[x][2]) +','+ str(goodnames[x][3]) +','+  goodgenes[x] + '\n'
  outgenes.append(gene)

with open(genesname, 'w') as g:
  for x in range(len(outgenes)):
    g.write(outgenes[x])
g.close()

print('Done.')
 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
The number of genes in "Features" = 6085
The "sub-1" Features line starts with:           563..2089
                     /gene="dnaA"
                     /locus_tag="PSF113_RS3023
The first 100 nucleotides = tgttacctggttcgtccacaacgggccggaatggcccccgttttaagagaccggggattctagagaaagcaagcctctaggtcaatttccaaccagcgtt
The number of annotated genes = 6084
['FOR', 'dnaA', 563, 2089, 1526, 0.0]
gtgtcagtggaactttggcagcagtgcgtagagcttttgcgcgatgagctgcctgcccaacaattcaacacttggatccgcccactacaggtcgaagccgaaggcgacgagttgcgtgtctatgcaccgaaccgtttcgttctcgactgggtcaacgaaaagtacctgggccgggtcatggaactgctcgacgagcatggcaatggcatggcgcctgcgctgtccttattaataggcagcaagcgcagctcggcaccgcgcgccgcacccaacgcgccgctggccgccgcagcgtcacaggcccagcagactgcaacggcaccggttaacaaccatgccgccccggcacccgcgcctgccccgagcaaacgcaacgcacaaaaggtcgccgaggtcagcgaagaaccctcccgggacagcttcgacccgatggccggtgccagctcccagcaggcgccggtgcgtgccgaacagcgcaccgtcc